# 1.导入必要的包

In [1]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 2.设置一些参数，如API Key和API网址等环境变量 **（可选）**

In [8]:
# os.environ["OPENAI_API_BASE"] = ""
# os.environ["OPENAI_API_KEY"] = ""

# 2.加载文章内容

In [3]:
with open("./content.md", "r", encoding="utf-8") as f:
    content = f.read()

In [4]:
# 检查加载的文档内容
print(len(content))  # 打印第一个文档内容的长度
print(content[:100])

8007
# 《黑神话：悟空》招贤纳士，游戏科学放出多个岗位
## 【本文由小黑盒作者@bgim于12月23日发布，转载请标明出处！】
随着《黑神话：悟空》爆火，游戏科学(Game Science)再次引发了玩


In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_text(content)

In [6]:
# 检查分割后的块数量和内容
print(len(all_splits))  # 打印分割后的文档块数量
print(len(all_splits[0]))  # 打印第一个块的字符数
print(all_splits[0])  # 打印第一个块的内容

10
807
# 《黑神话：悟空》招贤纳士，游戏科学放出多个岗位
## 【本文由小黑盒作者@bgim于12月23日发布，转载请标明出处！】
随着《黑神话：悟空》爆火，游戏科学(Game Science)再次引发了玩家的热烈讨论和关注。近期，游戏科学又发布了冬季招聘信息，宣告他们正在寻找各类人才加入团队，为这款备受期待的作品注入新的活力。 
除了程序员和设计师外，IP运营、网站开发等多个岗位也成为此次招聘的重点，显示出游戏科学不仅专注于技术开发，还在积极扩展《黑神话：悟空》的品牌和内容生态。这也意味着，玩家可以期待更多丰富的故事和角色设定。
对于热衷《黑神话：悟空》的玩家，现阶段正是加入这一创意团队的好机会。
《黑神话：悟空》的未来不仅仅局限于一款游戏，它的IP生态和世界观正在不断扩展，未来的内容和创意也将更加丰富。

# 游戏科学招人了!IP运营等岗位 扩展黑神话IP影响力?
## 2024-12-18 21:02:38 来源：游民星空 作者：猛鲨男鱼王 编辑：猛鲨男鱼王　浏览：15203
游戏科学在其官方渠道发布了2024年冬季的最新内部推荐岗位信息。此次招聘涵盖了杭州和深圳两地的多个职位。
岗位涉及了技术、设计、运营、行政等多个领域。其中可以看到对于IP运营和设计师的招聘需求，反映出游戏科学在原创IP开发和运营方面的战略布局，或许不久之后，我们可以看到更多《黑神话》IP的相关内容产出和亮相。

# “黑神话：悟空” 爆火，想去“游戏科学”上班，先来看看薪资待遇和要求
## 工程师老罗
### 一、公司背景
深圳市游科互动科技有限公司（游戏科学）是一家专注于探索互动娱乐体验的公司，其代表作品包括备受关注的《黑神话：悟空》。该公司致力于通过先进的技术和创新的理念，为玩家带来前所未有的游戏体验。

### 二、招聘岗位及薪资
根据最新的招聘信息，游戏科学在多个岗位上均有招聘需求，具体岗位及薪资范围如下：


# 3.使用 Chroma 向量存储和 OpenAIEmbeddings 模型，将分割的文档块嵌入并存储
### 经过测试，中转key有可能会有openai的embedding模型，只要在环境变量配置中转网址和key即可，但是生产力环境请使用官方key

In [9]:
vectorstore = Chroma.from_texts(
    texts=all_splits,
    embedding=OpenAIEmbeddings()
)

In [10]:
# 查看 vectorstore 数据类型
type(vectorstore)

langchain_chroma.vectorstores.Chroma

# 4.使用 VectorStoreRetriever 从向量存储中检索与查询最相关的文档

In [11]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [12]:
# 查看 type(retriever) 数据类型
print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [32]:
# 检查检索到的文档内容
retrieved_docs = retriever.invoke("游戏科学招聘地点在哪里？")
print(retrieved_docs[0])


page_content='# 《黑神话：悟空》招贤纳士，游戏科学放出多个岗位
## 【本文由小黑盒作者@bgim于12月23日发布，转载请标明出处！】
随着《黑神话：悟空》爆火，游戏科学(Game Science)再次引发了玩家的热烈讨论和关注。近期，游戏科学又发布了冬季招聘信息，宣告他们正在寻找各类人才加入团队，为这款备受期待的作品注入新的活力。 
除了程序员和设计师外，IP运营、网站开发等多个岗位也成为此次招聘的重点，显示出游戏科学不仅专注于技术开发，还在积极扩展《黑神话：悟空》的品牌和内容生态。这也意味着，玩家可以期待更多丰富的故事和角色设定。
对于热衷《黑神话：悟空》的玩家，现阶段正是加入这一创意团队的好机会。
《黑神话：悟空》的未来不仅仅局限于一款游戏，它的IP生态和世界观正在不断扩展，未来的内容和创意也将更加丰富。

# 游戏科学招人了!IP运营等岗位 扩展黑神话IP影响力?
## 2024-12-18 21:02:38 来源：游民星空 作者：猛鲨男鱼王 编辑：猛鲨男鱼王　浏览：15203
游戏科学在其官方渠道发布了2024年冬季的最新内部推荐岗位信息。此次招聘涵盖了杭州和深圳两地的多个职位。
岗位涉及了技术、设计、运营、行政等多个领域。其中可以看到对于IP运营和设计师的招聘需求，反映出游戏科学在原创IP开发和运营方面的战略布局，或许不久之后，我们可以看到更多《黑神话》IP的相关内容产出和亮相。

# “黑神话：悟空” 爆火，想去“游戏科学”上班，先来看看薪资待遇和要求
## 工程师老罗
### 一、公司背景
深圳市游科互动科技有限公司（游戏科学）是一家专注于探索互动娱乐体验的公司，其代表作品包括备受关注的《黑神话：悟空》。该公司致力于通过先进的技术和创新的理念，为玩家带来前所未有的游戏体验。

### 二、招聘岗位及薪资
根据最新的招聘信息，游戏科学在多个岗位上均有招聘需求，具体岗位及薪资范围如下：'


# 5.生成回答
定义 RAG 链，将用户问题与检索到的文档结合并生成答案
## 5.1 初始化大模型


In [14]:
# 很神奇，貌似在环境变量里填了中转API，就不用在初始化的时候填API了
llm = ChatOpenAI(model="gpt-4o-mini")

## 5.2 LangSmith hub提示词模板

In [33]:
# 使用 hub 模块拉取 rag 提示词模板
# 报错无关，人和程序都能跑 LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API   warnings.warn(
prompt = hub.pull("zbgd/rag-prompt")

D:\LLM\openai-quickstart\.venv\Lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [34]:
# 打印模板
print(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='你是问答任务的助手。 使用以下检索到的上下文来回答问题。 如果你不知道答案，就说你不知道。 最多使用三个句子并保持答案简洁。\nQuestion: {question} \nContext: {context} \nAnswer:'), additional_kwargs={})]


In [35]:
# 为 context 和 question 填充样例数据，并生成 ChatModel 可用的 Messages
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [36]:
# 查看提示词
print(example_messages[0].content)

你是问答任务的助手。 使用以下检索到的上下文来回答问题。 如果你不知道答案，就说你不知道。 最多使用三个句子并保持答案简洁。
Question: filler question 
Context: filler context 
Answer:


# 5.3 使用 LCEL 构建 RAG Chain


In [37]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 5.4 生成回答

In [38]:
# 流式生成回答
for chunk in rag_chain.stream("游戏科学招聘地点在哪里?"):
    print(chunk, end="", flush=True)

游戏科学的招聘地点主要在杭州和深圳两地。

In [39]:
# 同步生成回答
prompt_value = rag_chain.invoke("游戏科学下一步计划做什么?")

In [40]:
print(prompt_value)

游戏科学的下一步计划包括推出《黑神话：悟空》的DLC和开发基于中国神话的新游戏。此外，游戏科学还在扩招人才，以支持新项目的开发和公司规模的扩大。未来可能会有更多与《黑神话》相关的内容和IP生态的扩展。


In [48]:
# 批量生成回答
prompt_value = rag_chain.batch(["游戏科学接受了阿里巴巴的投资吗？"])
output_parser = StrOutputParser()
for idx, m in enumerate(prompt_value):
    print(f"{idx}:\n")
    print(output_parser.invoke(m))
    print("\n")

0:

我不知道游戏科学是否接受了阿里巴巴的投资。上下文中提到的投资主要是来自英雄游戏和腾讯。




## 5.5 自定义提示词模板

In [51]:
# 自定义提示词模板
from langchain_core.prompts import PromptTemplate
template = """
角色设定：你是一个优秀的猎头，能从复杂的公开信息中（如新闻中）筛选出招聘信息。
任务背景：{context}
任务目标: {question}
约束规则: 如果你不知道答案，就说"不知道"；让我们说中文。
"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [52]:
# 重新自定义 RAG Chain
custom_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)
prompt_value = custom_rag_chain.batch(["游戏科学招聘地点在哪里?", "游戏科学下一步计划做什么?", "游戏科学接受了阿里巴巴的投资吗？"])
output_parser = StrOutputParser()
for idx, m in enumerate(prompt_value):
    print(f"{idx}:")
    print(output_parser.invoke(m))
    print("\n")

0:
游戏科学的招聘地点包括深圳和杭州。


1:
游戏科学的下一步计划包括推出《黑神话：悟空》的DLC以及启动基于中国神话的新项目。这些新项目将不会再等待六年时间，而是希望能够更快地推进。此外，随着《黑神话：悟空》的成功，游戏科学还计划扩招，预计会有多个研发和运营岗位开放，以支持新项目的开发和品牌的扩展。


2:
不知道。




提问三个问题，使用langSmith hub提示词准确回答出来问题。使用自定义的提示词也成功回答出来问题，但是相比之下，提问向量数据库中未包含的信息，langSmith hub的提示词引导的ai回答还会给出相关的信息。我自己设置的提示词只会按照我的设定回答不知道。